In [1]:
import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { motion } from 'framer-motion';
import { format, startOfDay, isToday } from 'date-fns';
import { Link } from 'react-router-dom';
import { createPageUrl } from '@/utils';
import { Camera, TrendingUp, Utensils, Sparkles, ChevronRight } from 'lucide-react';
import { Button } from "@/components/ui/button";

import DailyProgress from '../components/dashboard/DailyProgress';
import MealHistory from '../components/dashboard/MealHistory';
import WeeklyChart from '../components/analytics/WeeklyChart';

export default function DashboardPage() {
  const [selectedMeal, setSelectedMeal] = useState(null);

  const { data: scans = [], isLoading: scansLoading } = useQuery({
    queryKey: ['foodScans'],
    queryFn: () => base44.entities.FoodScan.list('-created_date', 100),
  });

  const { data: userProfiles = [] } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];
  const todayScans = (scans || []).filter(scan => isToday(new Date(scan.created_date)));

  const getGreeting = () => {
    const hour = new Date().getHours();
    if (hour < 12) return 'Good morning';
    if (hour < 17) return 'Good afternoon';
    return 'Good evening';
  };

  const getMealSuggestion = () => {
    const hour = new Date().getHours();
    if (hour >= 6 && hour < 10) return "Time for breakfast?";
    if (hour >= 11 && hour < 14) return "Ready for lunch?";
    if (hour >= 17 && hour < 21) return "Dinner time?";
    return "Log a snack";
  };

  if (scansLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-2xl mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-8"
        >
          <p className="text-slate-500">{getGreeting()} 👋</p>
          <h1 className="text-2xl font-bold text-slate-800 mt-1">
            {format(new Date(), 'EEEE, MMMM d')}
          </h1>
        </motion.div>

        {/* Quick Scan CTA */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="mb-8"
        >
          <Link to={createPageUrl('Scan')}>
            <div className="bg-gradient-to-br from-emerald-500 to-teal-600 rounded-3xl p-6 text-white relative overflow-hidden group cursor-pointer">
              <div className="absolute top-0 right-0 w-32 h-32 bg-white/10 rounded-full -translate-y-1/2 translate-x-1/2" />
              <div className="absolute bottom-0 left-0 w-24 h-24 bg-white/10 rounded-full translate-y-1/2 -translate-x-1/2" />
              
              <div className="relative flex items-center justify-between">
                <div>
                  <p className="text-emerald-100 text-sm">{getMealSuggestion()}</p>
                  <p className="text-xl font-semibold mt-1">Scan your meal</p>
                </div>
                <motion.div 
                  whileHover={{ scale: 1.1 }}
                  className="w-14 h-14 bg-white/20 rounded-2xl flex items-center justify-center"
                >
                  <Camera className="w-7 h-7" />
                </motion.div>
              </div>
            </div>
          </Link>
        </motion.div>

        {/* Daily Progress */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.2 }}
          className="mb-8"
        >
          <DailyProgress todayScans={todayScans} targets={userProfile} />
        </motion.div>

        {/* Weekly Chart */}
        {scans.length > 0 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.3 }}
            className="mb-8"
          >
            <WeeklyChart scans={scans} targets={userProfile} />
          </motion.div>
        )}

        {/* AI Insights */}
        {todayScans.length >= 2 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.4 }}
            className="mb-8"
          >
            <div className="bg-gradient-to-br from-violet-50 to-purple-50 rounded-3xl p-6 border border-violet-100">
              <div className="flex items-start gap-4">
                <div className="w-12 h-12 rounded-2xl bg-gradient-to-br from-violet-500 to-purple-600 flex items-center justify-center flex-shrink-0">
                  <Sparkles className="w-6 h-6 text-white" />
                </div>
                <div>
                  <h3 className="font-semibold text-slate-800 mb-1">Daily Insight</h3>
                  <p className="text-sm text-slate-600">
                    {(() => {
                      const totalProtein = todayScans.reduce((sum, s) => sum + (s.protein || 0), 0);
                      const proteinTarget = userProfile?.daily_protein_target || 150;
                      if (totalProtein < proteinTarget * 0.5) {
                        return `You're at ${Math.round(totalProtein)}g protein. Consider adding a protein-rich food to hit your ${proteinTarget}g goal!`;
                      }
                      const avgHealthScore = todayScans.reduce((sum, s) => sum + (s.health_score || 5), 0) / todayScans.length;
                      if (avgHealthScore >= 7) {
                        return "Great choices today! Your meals have been nutritious and balanced.";
                      }
                      return "Try adding more vegetables or whole foods to boost your nutrition score.";
                    })()}
                  </p>
                </div>
              </div>
            </div>
          </motion.div>
        )}

        {/* Meal History */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.5 }}
        >
          <div className="flex items-center justify-between mb-4">
            <h2 className="text-lg font-semibold text-slate-800">Meal History</h2>
            <Link to={createPageUrl('History')}>
              <Button variant="ghost" size="sm" className="text-slate-500">
                View all
                <ChevronRight className="w-4 h-4 ml-1" />
              </Button>
            </Link>
          </div>
          <MealHistory 
            scans={scans.slice(0, 10)} 
            onSelectMeal={setSelectedMeal}
          />
        </motion.div>
      </div>
    </div>
  );
}

SyntaxError: invalid character '👋' (U+1F44B) (3417284579.py, line 63)